# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [16]:
# passo 1: entrar na internet e pegar as cotações
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

# passo 2: pegar a cotação do dólar
# entrar no google
navegador.get("https://www.google.com/")
# digitar "cotação do dólar"
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação dólar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# copiar o resultado da cotação do dólar
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)

# passo 3: pegar a cotação do euro
# entrar no google
navegador.get("https://www.google.com/")
# digitar "cotação do dólar"
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# copiar o resultado da cotação do dólar
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# passo 4: pegar a cotação do ouro
# entrar no site do melhor câmbio
navegador.get("https://www.melhorcambio.com/ouro-hoje")
#pegar a cotação
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()

5.22
6.128541
294.12


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [17]:
# passo 5: importar e atualizar a base de dados
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [18]:
# atualizar a cotação
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)

tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)

tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço de compra -> preço original x cotação
tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]

# atualizar o preço de venda -> preõ de compra x margem
tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Margem"]
tabela["Preço Final"] = tabela["Preço Final"].map("{:.2f}".format)

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.220000,5219.9478,1.40,7307.93
1,Carro Renault,4500.00,Euro,6.128541,27578.4345,2.00,55156.87
2,Notebook Dell,899.99,Dólar,5.220000,4697.9478,1.70,7986.51
3,IPhone,799.00,Dólar,5.220000,4170.7800,1.70,7090.33
4,Carro Fiat,3000.00,Euro,6.128541,18385.6230,1.90,34932.68
5,Celular Xiaomi,480.48,Dólar,5.220000,2508.1056,2.00,5016.21
6,Joia 20g,20.00,Ouro,294.120000,5882.4000,1.15,6764.76


### Agora vamos exportar a nova base de preços atualizada

In [20]:
# passo 6: exportar a base de dados atualizada
tabela.to_excel("Produtos Novo.xlsx", index=False)